# CmdStanPy Tutorial


### Workflow Outline

Given a dataset and a model specification written as a Stan program, the CmdStanPy workflow is:

#### Assemble input data as either:
  + a Python `dict` object consisting of key-value pairs where the key corresponds
 to Stan data variables and the value is of the correct type and shape.
  + an existing data file on disk in either JSON or Rdump format.
  + CmdStanPy provides methods for converting former to latter.




#### Compile Stan program - function `compile_model`
  + takes as input a Stan program and produces the corresponding c++ executable.
  + both Stan program and c++ executable exist as on-disk files

#### Assemble input data as either:
  + a Python `dict` object consisting of key-value pairs where the key corresponds
 to Stan data variables and the value is of the correct type and shape.
  + an existing data file on disk in either JSON or Rdump format.

#### Run sampler - function `sample`
  + invokes Stan's NUTS-HMC sampler to condition model on input data, produces output csv files
  + runs any number of chains - should run at least 2 (default)
  + lightweight object `RunSet` keeps track of sampler arguments, per-chain outcome, output files
  + returns `PosteriorSample` object which contains information about sample

#### Analyze resulting `PosteriorSample`
  + `PosteriorSample` contains names of csv output files, names of csv column headers
  + attribute `sample` assembles in-memory sample from csv files
  + methods `summary` and `diagnose` invoke CmdStan tools `bin/stansummary` and `bin/diagnose` respectively


### Installation

* Download this repo from GitHub

* run script `make_cmdstan.sh` - (need equivalent for Windows) - this installs and builds cmdstan in subdirectory `releases`




### Example 1:  example model `bernoulli.stan`, data `bernoulli.data.json`

Import relevant classes and methods


In [1]:
import os
import os.path
from cmdstanpy.lib import Model, PosteriorSample
from cmdstanpy.cmds import compile_model, sample

The CmdStanPy directory `test/data` contains test data and models


In [ ]:
datafiles_path = os.path.expanduser(
    os.path.join("~", "github", "stan-dev", "cmdstanpy", "test", "data"))
stan = os.path.join(datafiles_path, "bernoulli.stan")
jdata = os.path.join(datafiles_path, "bernoulli.data.json")

Call `compile_model` with path to Stan program, returns `Model` object `bernoulli_model`

In [ ]:
bernoulli_model = compile_model(stan)
print(bernoulli_model)

Call `sample` to run NUTS-HMC sampler, returns `PosteriorSample` object `a_sample`

In [ ]:
a_sample = sample(bernoulli_model, chains=4, cores=2, data_file=jdata)

In [ ]:
print(a_sample)

### `PosteriorSample` object functions and attributes

#### PosteriorSample.summary()

The `summary` function returns output of CmdStan bin/stansummary as pandas.DataFrame:

In [ ]:
a_sample.summary()

#### PosteriorSample.diagnose()

The `diagnose` function prints diagnostics to console:

In [ ]:
a_sample.diagnose()

#### PosteriorSample.sample

The `sample` property is a 3-D numpy ndarray which contains all draws across all chains.  The ndarray is stored column major format so that values for each parameter are stored contiguously in memory, likewise all draws from a chain are contiguous.  Thus the dimensions of the ndarray are arranged as follows:  (draws, chains, columns):

In [ ]:
a_sample.sample, a_sample.sample.shape

#### PosteriorSample.extract()

The `extract` function flattens the `sample` 3-dimensional ndarray (draws X chains X columns) to a  pandas.DataFrame, (i.e., a 2-dimensional ndarray), one draw per row.  

In [ ]:
samp = a_sample.extract()
samp.shape, samp.columns, samp.theta.plot.density()

In [ ]:
thetas = a_sample.extract(params=['theta'])

In [5]:
import tempfile
from cmdstanpy import TMPDIR
from cmdstanpy.lib import StanData
fd = tempfile.NamedTemporaryFile(mode='w+', suffix='.json', dir=TMPDIR, delete=False)
data_file = fd.name
print(data_file)

/var/folders/sc/0f0wdc_11_xgjs2v52g20fvr0000gn/T/tmp72tl0rbq/tmpkoi0d50y.json


In [6]:
data_dict = { "N" : 10, "y" : [0,1,0,0,0,0,0,0,0,1] }
sd = StanData(data_file)

In [7]:
sd.write_json(data_dict)

In [9]:
import json
with open(data_file, 'w') as fd:
    json.dumps(data_dict)

json.dumps(data_dict)

'{"N": 10, "y": [0, 1, 0, 0, 0, 0, 0, 0, 0, 1]}'